In [1]:
import numpy as np  
import navpy
import math
import pandas as pd 
import os, csv, sys
import glob
import subprocess
from tqdm.notebook import tqdm
from dataclasses import dataclass
from scipy.interpolate import InterpolatedUnivariateSpline
from datetime import datetime, timezone, timedelta

folder_path = os.path.abspath('gnss_analysis')
sys.path.append(folder_path)
from gnssutils import EphemerisManager,WLS,Troposphere,Ionosphere,ElevationAzimuthAngle

In [2]:
path = 'device_gnss_2021.csv'
gnss = pd.read_csv(path, index_col=False)
selected_col = ['utcTimeMillis','TimeNanos','LeapSecond','TimeUncertaintyNanos','FullBiasNanos','BiasNanos','BiasUncertaintyNanos','DriftNanosPerSecond','DriftUncertaintyNanosPerSecond','HardwareClockDiscontinuityCount','Svid','TimeOffsetNanos','State','ReceivedSvTimeNanos','ReceivedSvTimeUncertaintyNanos','Cn0DbHz','PseudorangeRateMetersPerSecond','PseudorangeRateUncertaintyMetersPerSecond','AccumulatedDeltaRangeState','AccumulatedDeltaRangeMeters','AccumulatedDeltaRangeUncertaintyMeters','CarrierFrequencyHz','CarrierCycles','CarrierPhase','CarrierPhaseUncertainty','MultipathIndicator','SnrInDb','ConstellationType','AgcDb','BasebandCn0DbHz','FullInterSignalBiasNanos','FullInterSignalBiasUncertaintyNanos','SatelliteInterSignalBiasNanos','SatelliteInterSignalBiasUncertaintyNanos','CodeType','ChipsetElapsedRealtimeNanos','ArrivalTimeNanosSinceGpsEpoch','RawPseudorangeMeters','RawPseudorangeUncertaintyMeters','SignalType','ReceivedSvTimeNanosSinceGpsEpoch']
measurement = gnss[selected_col]

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'

measurement['utcTimeMillis'] = pd.to_numeric(measurement['utcTimeMillis'])
measurement['Cn0DbHz'] = pd.to_numeric(measurement['Cn0DbHz'])
measurement['TimeNanos'] = pd.to_numeric(measurement['TimeNanos'])
measurement['FullBiasNanos'] = pd.to_numeric(measurement['FullBiasNanos'])
measurement['ReceivedSvTimeNanos']  = pd.to_numeric(measurement['ReceivedSvTimeNanos'])
measurement['PseudorangeRateMetersPerSecond'] = pd.to_numeric(measurement['PseudorangeRateMetersPerSecond'])
measurement['ReceivedSvTimeUncertaintyNanos'] = pd.to_numeric(measurement['ReceivedSvTimeUncertaintyNanos'])
measurement['BiasNanos'] = pd.to_numeric(measurement['BiasNanos']).astype(float)
measurement['TimeOffsetNanos'] = pd.to_numeric(measurement['TimeOffsetNanos'])
measurement['Svid'] = pd.to_numeric(measurement['Svid'])
measurement['ArrivalTimeNanosSinceGpsEpoch'] = pd.to_numeric(measurement['ArrivalTimeNanosSinceGpsEpoch'])
measurement['ReceivedSvTimeNanosSinceGpsEpoch'] = pd.to_numeric(measurement['ReceivedSvTimeNanosSinceGpsEpoch'])
measurement['RawPseudorangeMeters'] = pd.to_numeric(measurement['RawPseudorangeMeters'])


In [4]:
# Lọc các vệ tinh thuộc GPS (ConstellationType == 1)
# Loc cac ve tinh co Multipath
measurement.loc[measurement['ConstellationType'] == 1, 'Constellation'] = 'G'

measurement = measurement.loc[measurement['MultipathIndicator'] == 0]

measurement = measurement.loc[measurement['ReceivedSvTimeUncertaintyNanos'] < 500]

# Chỉ lấy các vệ tinh bắt GPS
measurement = measurement.loc[measurement['Constellation'] == 'G']
# Đảm bảo cột 'Svid' là chuỗi ký tự để có thể sử dụng hàm str.len()
measurement['Svid'] = measurement['Svid'].astype(str)
# Thêm số 0 vào trước 'Svid' nếu chiều dài của nó là 1
measurement.loc[measurement['Svid'].str.len() == 1, 'Svid'] = '0' + measurement['Svid']
# Tạo cột mới 'SvName' bằng cách nối 'Constellation' và 'Svid'
measurement['SvName'] = measurement['Constellation'] + measurement['Svid']

measurement = measurement.loc[measurement['CarrierFrequencyHz'] > 1275420030]

measurement = measurement.reset_index(drop=True)

In [5]:
WeekNanos = 604800000000000

from decimal import Decimal, getcontext, ROUND_DOWN

measurement['TimeNanos'] = measurement['TimeNanos'].apply(Decimal)
measurement['FullBiasNanos'] = measurement['FullBiasNanos'].apply(Decimal)
measurement['BiasNanos'] = measurement['BiasNanos'].apply(lambda x: Decimal(x).quantize(Decimal('0.001'), rounding=ROUND_DOWN))

# Tính toán giá trị
measurement['ArrivalTimeNanosSinceGpsEpoch_1'] = (measurement['TimeNanos'] - measurement['FullBiasNanos'] - measurement['BiasNanos'])

res = measurement['ArrivalTimeNanosSinceGpsEpoch_1'].iloc[0]%1000000000

measurement['ArrivalTimeNanosSinceGpsEpoch_1'] = measurement['ArrivalTimeNanosSinceGpsEpoch_1'].apply(lambda x: round(x, -9)) + res

measurement['GpsWeekNum'] = np.floor(measurement['ArrivalTimeNanosSinceGpsEpoch_1']/WeekNanos).astype(int)
measurement['ReceivedSvTimeNanosSinceGpsEpoch_1'] = measurement['GpsWeekNum'] * WeekNanos + measurement['ReceivedSvTimeNanos']
measurement['ReceivedSvTimeNanosSinceGpsEpoch_1'].apply(Decimal)
pd.set_option('display.float_format', lambda x: '%.16f' % x)
measurement['ArrivalTimeNanosSinceGpsEpoch_1']

0        1310762975441209060.922
1        1310762975441209060.922
2        1310762975441209060.922
3        1310762975441209060.922
4        1310762975441209060.922
                  ...           
17728    1310764871441209060.922
17729    1310764871441209060.922
17730    1310764871441209060.922
17731    1310764871441209060.922
17732    1310764871441209060.922
Name: ArrivalTimeNanosSinceGpsEpoch_1, Length: 17733, dtype: object

In [6]:
LightSpeed = 0.299792458
measurement['RawPseudorangeMeters_1'] = (measurement['ArrivalTimeNanosSinceGpsEpoch_1'] - measurement['ReceivedSvTimeNanosSinceGpsEpoch_1']) * Decimal(LightSpeed)
measurement['RawPseudorangeUncertaintyMeters_1'] = measurement['ReceivedSvTimeUncertaintyNanos'] * LightSpeed

In [9]:
selected_columns = measurement[['ArrivalTimeNanosSinceGpsEpoch_1','ArrivalTimeNanosSinceGpsEpoch', 'ReceivedSvTimeNanosSinceGpsEpoch_1','ReceivedSvTimeNanosSinceGpsEpoch', 'RawPseudorangeMeters_1', 'RawPseudorangeMeters']]
pd.set_option('display.float_format', lambda x: f'{x:.0f}' if x == int(x) else f'{x}')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print(selected_columns.head(100))

   ArrivalTimeNanosSinceGpsEpoch_1  ArrivalTimeNanosSinceGpsEpoch  \
0          1310762975441209060.922            1310762975441209088   
1          1310762975441209060.922            1310762975441209088   
2          1310762975441209060.922            1310762975441209088   
3          1310762975441209060.922            1310762975441209088   
4          1310762975441209060.922            1310762975441209088   
5          1310762975441209060.922            1310762975441209088   
6          1310762975441209060.922            1310762975441209088   
7          1310762976441209060.922            1310762976441209088   
8          1310762976441209060.922            1310762976441209088   
9          1310762976441209060.922            1310762976441209088   
10         1310762976441209060.922            1310762976441209088   
11         1310762976441209060.922            1310762976441209088   
12         1310762976441209060.922            1310762976441209088   
13         1310762976441209060.922